# GeoParquet, parquet for geospatial data

In this tutorial, we will learn what are the parquet and geoparquet file format.

## 1. What is apache parquet?

**Apache Parquet** is a `columnar storage file format` widely used in big data analytics (Spark, Hive, Dask, DuckDB). Unlike row-based file format, parquet stored values from the same column sequentially. This makes column scans fast and compresses better.

For example, row based storage will save data like:
```csv
id, name, age
1, Alice, 34
2, Bob, 45
3, Carol, 29
```

And, column based storage will save data like:
```csv
id: [1, 2, 3]
name: [Alice, Bob, Carol]
age: [34, 45, 29]
```

Compare to row based format, `columnar storage file format (parquet)` has the below advantages are:
- Column encoding is possible: the encoding such as `dictionary`, `run-length encoding (RLE)`, `bit-packing` can improve storage and lecture speed.
- Better compression: values are similar in the same column, so it compresses much better than row formats.
- Better data loading performance: `projection pushdown`(reads only required column), `predicate pushdown`(reads only relevant chunks if filter conditions match).
- The Schema and metadata are embedded and customizable.
- Data partition is supported: works well in distributed systems, and efficient for large-scale analytical queries (data warehouses, data lakes).


The disadvantages are:
- Not Ideal for datasets(<1MB): metadata and encoding may cost you more than the actual data
- Not human-readable: Parquet is a binary format. You need tools (e.g. Pandas, DuckDB, etc.) to read and debug.
Complexity
- Write Cost: Writing is more CPU-intensive (encoding + compression). Insert or update a row may require to rewrite the whole parquet file.

> Parquet is a file format designed for large-scale data analytics with the pattern Write Once Read Many(worm). It's not ideal for small, transactional workloads(e.g. need update data every second).

## 2. What is GeoParquet?

**GeoParquet** is an extension of `Apache Parquet` designed for storing `geospatial vector data` (points, lines, polygons) in a columnar, compressed, efficient format.

The official website of the GeoParquet project is [here](https://geoparquet.org/)

To be able to store `geospatial vector data`, **GeoParquet** adds the below concepts in the metadata of the parquet:
- which column is geometry column
- how geometries data is encoded(e.g. WKB, GeoArrow, etc.).
- which CRS(Coordinate Reference System) is used.

For example, a typical geoparquet metadata looks like

```json
{
  "version": "1.0.0-beta.1",
  "primary_column": "geometry",
  "columns": {
    "geometry": {
      "encoding": "WKB",
      "geometry_types": ["Polygon", "MultiPolygon"],
      "crs": "EPSG:4326"
    }
  }
}

```

> The parquet file has one geometry column called `geometry`, it use `WKB` as geospatial data encoding. The crs is
> `EPSG:4326`


The advantage of geoparquet:

- Much faster than GeoJSON or Shapefile for analytics.
- Can be partitioned for big data volume. It works well with distributed engines (Spark, Dask).
- Reduce disk usage due to columnar storage & compression.
- Standardized metadata ensures interoperability.

### 2.1 Supported tools

The below tools can read geoparquet

- GeoPandas (Python)
- Sedona(Python, Java, scala)
- DuckDB(Python, R, SQL)
- Fiona/GDAL
- QGIS/ArcGIS Pro

## 3. Read a geoparquet file with sedona

In [1]:
from sedona.spark import *
from pyspark.sql import SparkSession, DataFrame
from pathlib import Path
from pyspark.sql.functions import trim, split, expr, col

In [ ]:
import os
os.environ["PYSPARK_PYTHON"]="python"
os.environ["PYSPARK_DRIVER_PYTHON"]="python"

In [2]:
# build a sedona session offline
project_root_dir = Path.cwd().parent
print(project_root_dir.as_posix())

C:/Users/PLIU/Documents/git/Seminar_PySpark_Sedona_GeoParquet


In [3]:
# here we choose sedona 1.7.2 for spark 3.5.* build with scala 2.12
jar_folder = Path(f"{project_root_dir}/jars/sedona-35-212-172")
jar_list = [str(jar) for jar in jar_folder.iterdir() if jar.is_file()]
jar_path = ",".join(jar_list)

# build a sedona session (sedona = 1.7.2) offline
spark = SparkSession.builder \
    .appName("sedona_tutorial") \
    .master("local[*]") \
    .config("spark.jars", jar_path) \
    .getOrCreate()

In [4]:
# create a sedona context
sedona = SedonaContext.create(spark)

In [5]:
sc = spark.sparkContext
# use utf as default encoding
sc.setSystemProperty("sedona.global.charset", "utf8")

In [10]:
data_dir = f"{project_root_dir}/data"
airports_parquet_path = f"{data_dir}/airports.parquet"

raw_parquet_df= sedona.read.format("geoparquet").load(airports_parquet_path)

In [11]:
raw_parquet_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- scalerank: string (nullable = true)
 |-- featurecla: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- abbrev: string (nullable = true)
 |-- location: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- wikipedia: string (nullable = true)
 |-- natlscale: string (nullable = true)



In [12]:
geo_airports = raw_parquet_df.select("geometry","name","iata_code","gps_code")
geo_airports.show(1,truncate=False,vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 geometry  | POINT (113.93501638737635 22.315332828086753)                                                                                                                                                                                                                  
 name      | Hong Kong Int'l                                                                                                                                                                                                                                                
 iata_code | HKG                                                                                                                                                                                 

## 4. Write a geoparquet file

Let's try to write a geoparquet file with default metadata.

In [14]:
out_dir = f"{data_dir}/tmp"
simple_airports_out_path = f"{out_dir}/simple_geo_airports_parquet"

In [15]:
geo_airports.write.format("geoparquet").option("geoparquet.version","1.1.0").save(simple_airports_out_path)

In [18]:
# now let's read the metadata of the generated geoparquet file
simple_df_metadata = sedona.read.format("geoparquet.metadata").load(simple_airports_out_path)

In [19]:
simple_df_metadata.printSchema()

root
 |-- path: string (nullable = true)
 |-- version: string (nullable = true)
 |-- primary_column: string (nullable = true)
 |-- columns: map (nullable = true)
 |    |-- key: string
 |    |-- value: struct (valueContainsNull = true)
 |    |    |-- encoding: string (nullable = true)
 |    |    |-- geometry_types: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- bbox: array (nullable = true)
 |    |    |    |-- element: double (containsNull = true)
 |    |    |-- crs: string (nullable = true)
 |    |    |-- covering: string (nullable = true)



In [20]:
simple_df_metadata.show(truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 path           | file:/C:/Users/PLIU/Documents/git/Seminar_PySpark_Sedona_GeoParquet/data/tmp/simple_geo_airports_parquet/part-00000-f0b49b97-1b27-4a02-a829-bfe69fa0b248-c000.snappy.parquet 
 version        | 1.1.0                                                                                                                                                                        
 primary_column | geometry                                                                                                                                                                     
 columns        | {geometry -> {WKB, [Point], [-175.135635, -53.005069825517666, 178.5600483699593, 71.289299], null, NULL}}                                                                   



### 4.1 Understand the geoparquet metadata

The GeoParquet metadata stores the following information:
- path: The file path
- version: geoparquet version(different version has different metadata schema)
- primary_column: The bbox index is calculated based on the primary column.
- columns: a key value map which stores column metadata for each geometry column. It contains the below information for each column:
    - geometry_type: type of geometric object like point or polygon
    - bbox: bounding box of objects in file
    - crs: Coordinate Reference System
    - covering: a struct column of xmin, ymin, xmax, ymax that provides the row group statistics for GeoParquet 1.1 files

#### 4.1.1 Bounding box (bbox) in GeoParquet

The `bounding box(bbox)` metadata specifies the area covered by geometric shapes in a given file. Suppose you query points in a region not covered by the bounding box for a given file. The engine(sedona) can `skip that entire file` when executing the query because it’s known that it does not cover any relevant data.

Skipping entire files of data can massively improve performance. The more data that’s skipped, the faster the query will run.

Let’s look at an example of a dataset with three bounding boxes. Imagine we have a dataset with three partitions/files, and each partition/file has an associated bbox
![bbox1](../assets/geoparquet_bbox1.png)

Now, let’s apply a `spatial filter(ST_Within)` to read points within a particular area(e.g. bbox2).
Sedona will only read the data in partition/file of bbox2, the other two files will be omitted.
![bbox2](../assets/geoparquet_bbox2.png)

### 4.2 Custom metadata in geo parquet

As parquet allows us to insert custom metadata, so geo parquet has this ability too.

The most used metadata in geo parquet is:
  - crs coordinates for geometry column
  - covering metadata: For each geometry column, a top-level struct column containing xmin, ymin, xmax,ymax. The `covering` field specifies a bounding box column to help accelerate spatial data retrieval


#### 4.2.1 Integrated CRS coordinates for geometry columns

The maps of the surface of the earth is in two dimensions. But, as you know, the world is actually a three-dimensional globe. So we have to use a method called a `map projection` to render it as a flat surface. We use a coordinate reference system (CRS) to show how the projected points correspond to real locations on Earth.

For example, the coordinate (Latitude: 48.8566, Longitude: 2.3522) is in Paris in the `WGS84(ESPG:4326)` CRS. The same point in `EPSG:3857` (e.g. OSM) would be (X: 261845.83,Y: 6250561.88)

In [22]:
crs_code_for_geometry ="""
{
    "$schema": "https://proj.org/schemas/v0.7/projjson.schema.json",
    "type": "GeographicCRS",
    "name": "WGS 84",
    "datum_ensemble": {
        "name": "World Geodetic System 1984 ensemble",
        "members": [
            {
                "name": "World Geodetic System 1984 (Transit)",
                "id": {
                    "authority": "EPSG",
                    "code": 1166
                }
            },
            {
                "name": "World Geodetic System 1984 (G730)",
                "id": {
                    "authority": "EPSG",
                    "code": 1152
                }
            },
            {
                "name": "World Geodetic System 1984 (G873)",
                "id": {
                    "authority": "EPSG",
                    "code": 1153
                }
            },
            {
                "name": "World Geodetic System 1984 (G1150)",
                "id": {
                    "authority": "EPSG",
                    "code": 1154
                }
            },
            {
                "name": "World Geodetic System 1984 (G1674)",
                "id": {
                    "authority": "EPSG",
                    "code": 1155
                }
            },
            {
                "name": "World Geodetic System 1984 (G1762)",
                "id": {
                    "authority": "EPSG",
                    "code": 1156
                }
            },
            {
                "name": "World Geodetic System 1984 (G2139)",
                "id": {
                    "authority": "EPSG",
                    "code": 1309
                }
            }
        ],
        "ellipsoid": {
            "name": "WGS 84",
            "semi_major_axis": 6378137,
            "inverse_flattening": 298.257223563
        },
        "accuracy": "2.0",
        "id": {
            "authority": "EPSG",
            "code": 6326
        }
    },
    "coordinate_system": {
        "subtype": "ellipsoidal",
        "axis": [
            {
                "name": "Geodetic latitude",
                "abbreviation": "Lat",
                "direction": "north",
                "unit": "degree"
            },
            {
                "name": "Geodetic longitude",
                "abbreviation": "Lon",
                "direction": "east",
                "unit": "degree"
            }
        ]
    },
    "scope": "Horizontal component of 3D system.",
    "area": "World.",
    "bbox": {
        "south_latitude": -90,
        "west_longitude": -180,
        "north_latitude": 90,
        "east_longitude": 180
    },
    "id": {
        "authority": "EPSG",
        "code": 4326
    }
}"""

In [23]:
geoparquet_crs_sample_path = f"{out_dir}/geoparquet_custom_crs_sample"


geo_airports.write.mode("overwrite").format("geoparquet").option("geoparquet.crs.geometry",crs_code_for_geometry).option("geoparquet.version","1.1.0").save(geoparquet_crs_sample_path)

In [24]:
geo_crs_meta = sedona.read.format("geoparquet.metadata").load(geoparquet_crs_sample_path)

geo_crs_meta.show(truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### 4.2.2 Covering metadata (Column level bbox)

Since `v1.6.1`, Sedona supports writing the **covering field** to geometry column metadata. The covering field specifies a bounding box column to help accelerate spatial data retrieval.

The bounding box column should be a top-level struct column containing `xmin, ymin, xmax, ymax` columns. If the DataFrame you are writing contains such columns, you can specify
`.option("geoparquet.covering.<geometryColumnName>", "<coveringColumnName>")` option to write covering metadata to GeoParquet files

In [25]:
geoparquet_column_bbox_sample_path = f"{out_dir}/geoparquet_column_bbox_sample"


df_with_bbox= geo_airports.withColumn("bbox",  expr("struct(ST_XMin(geometry) AS xmin, ST_YMin(geometry) AS ymin, ST_XMax(geometry) AS xmax, ST_YMax(geometry) AS ymax)"))
df_with_bbox.show(5, truncate=False)

+---------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|geometry                                     |name                                                                                                             

In [26]:
df_with_bbox.write.format("geoparquet").option("geoparquet.covering.geometry", "bbox").save(geoparquet_column_bbox_sample_path)

In [27]:
geo_bbox_meta = sedona.read.format("geoparquet.metadata").load(geoparquet_column_bbox_sample_path)

geo_bbox_meta.show(truncate=False, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 path           | file:/C:/Users/PLIU/Documents/git/Seminar_PySpark_Sedona_GeoParquet/data/tmp/geoparquet_column_bbox_sample/part-00000-58f6c6bd-239d-4f15-9052-d1d1c2d6bf14-c000.snappy.parquet                               
 version        | 1.1.0                                                                                                                                                                                                        
 primary_column | geometry                                                                                                                                                                                                     
 columns        | {geometry -> {WKB, [Point], [-175.135635, -53.005069825517666, 178.5600483699593, 71.2

In [ ]:
# close the spark session
spark.stop()

# Appendix

## Coordinate Reference System (CRS)

The maps of the surface of the earth is in two dimensions. But, as you know, the world is actually a three-dimensional globe. So we have to use a method called a `map projection` to render it as a flat surface. We use a coordinate reference system (CRS) to show how the projected points correspond to real locations on Earth.

A Coordinate Reference System (CRS) is fundamental in geospatial data and GIS (Geographic Information Systems) because it defines how spatial data is represented on the Earth's surface. Without a proper CRS, geographic data from different sources would not align correctly, leading to errors and confusion in analysis, visualization, and decision-making.

## **WGC**(World Geodetic reference System)

It's used globally for mapping and navigation. The most widely known version is **WGS84**, which is the standard used by GPS. It defines the shape of the Earth (ellipsoid), the datum, and the coordinate system.

Key Features:

  - Datum: WGS84 Datum, a global standard for latitude, longitude, and ellipsoidal height.
  - Coordinate System: A geographic coordinate system (latitude/longitude) on the WGS84 ellipsoid.
   - Usage: Primarily used for GPS and satellite navigation systems.
      -
 > WGS84 is often represented as EPSG:4326 in the EPSG registry, a geographic CRS with latitude and longitude coordinates.


## OGC(Open Geospatial Consortium)

**OGC** is an international organization that sets standards for geospatial and location-based services. It doesn’t define specific CRS codes but provides frameworks and specifications for managing and using CRS in geospatial systems.

The OGC CRS URNs are part of its standard for referencing coordinate systems.

```text
urn:ogc:def:crs:EPSG::4326
```

## EPSG(European Petroleum Survey Group)

**EPSG** is a registry maintained by the `International Association of Oil & Gas Producers (IOGP)`. It provides a database of `standardized codes for various CRS definitions`, including geographic, projected, and vertical coordinate systems.

EPSG codes are numeric identifiers for specific CRS or transformation. For example:
 - **EPSG:4326**: `WGS84` in geographic coordinates (latitude/longitude).
 - **EPSG:3857**: `WGS84` Web Mercator projection (used by most web mapping services).

> Widely used in GIS software (e.g., QGIS, ArcGIS) and geospatial APIs.


